## Housekeeping 

In [ ]:
# !pip install pyarrow
# !pip install -U datasets huggingface_hub fsspec


In [38]:
# !pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.2 MB/s eta 0:00:00


In [ ]:
%%time

!conda update conda && conda install -y --file environment.yml

In [ ]:
!conda update -n base -c conda-forge conda

In [2]:
# # HF token was created and saved in file; should be able to call API
# import os
# token_dir = os.path.expanduser("~/.huggingface")
# os.makedirs(token_dir, exist_ok=True)
# with open(os.path.join(token_dir, "token"), "w") as f: 
#     f.write("hf_...")

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          37G   27M   37G   1% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.8G     0  7.8G   0% /sys/fs/cgroup
shm             1.8G   40K  1.8G   1% /dev/shm
/dev/nvme0n1p1  180G   59G  122G  33% /usr/bin/nvidia-smi
/dev/nvme2n1     20G   14G  6.3G  69% /home/sagemaker-user
/dev/nvme1n1    233G   14G  219G   6% /mnt/sagemaker-nvme
tmpfs           7.8G   12K  7.8G   1% /proc/driver/nvidia
tmpfs           7.8G  784K  7.8G   1% /run/nvidia-persistenced/socket
tmpfs           7.8G     0  7.8G   0% /proc/acpi
tmpfs           7.8G     0  7.8G   0% /sys/firmware


In [3]:
# custom cache directory, must run before import load_dataset
# rerun everytime you load datasets, persists only in kernel 
# import os
# new_cache_dir = "/home/sagemaker-user/dialect_model/data"
# os.makedirs(new_cache_dir, exist_ok=True)
# os.environ["HF_DATASETS_CACHE"] = new_cache_dir

# using temp cache for downloading large datasets, data is ephermeral and deletes when instance is stopped 
import os
os.makedirs("/mnt/sagemaker-nvme/hf_datasets_cache", exist_ok=True)
os.environ["HF_DATASETS_CACHE"] = "/mnt/sagemaker-nvme/hf_datasets_cache"


In [4]:
#import needed libraries
from datasets import load_dataset, Audio
import torch
from tqdm import tqdm
import pandas as pd
import time
import os
import pickletools

## Load Data from HF

In [5]:
try:
    shanghai_corpus = load_dataset("TingChen-ppmc/Shanghai_Dialect_Conversational_Speech_Corpus", split = "train", streaming = True) #3.79k rows
    print("Shanghai corpus loaded successfully")
except Exception as e:
    print(f"Error loading Shanghai corpus: {e}") 
    shanghai_corpus = None

Shanghai corpus loaded successfully


In [6]:
try:
    mandarin_corpus = load_dataset("urarik/free_st_chinese_mandarin_corpus", split="test", streaming = True) #10k rows 
    print("Mandarin corpus loaded successfully")
except Exception as e:
    print(f"Error loading Mandarin corpus: {e}")
    mandarin_corpus = None

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Mandarin corpus loaded successfully


In [7]:
try:
    sichuan_corpus = load_dataset("wanghaikuan/sichuan", split='train', streaming=True) #6k rows 
    print("Sichuan corpus loaded successfully")
except Exception as e:
    print(f"Error loading Sichuan corpus: {e}")
    sichuan_corpus = None


Sichuan corpus loaded successfully


In [8]:
try:
    cantonese_corpus = load_dataset("ziyou-li/cantonese_daily", split="train", streaming=True) #4k rows
    print("Cantonese corpus loaded successfully")
except Exception as e:
    print(f"Error loading Cantonese corpus: {e}")
    cantonese_corpus = None

Resolving data files:   0%|          | 0/4061 [00:00<?, ?it/s]

Cantonese corpus loaded successfully


In [9]:
cantonese_corpus

IterableDataset({
    features: ['audio', 'sentence'],
    num_shards: 1
})

In [10]:
mandarin_corpus

IterableDataset({
    features: ['audio', 'sentence'],
    num_shards: 3
})

In [11]:
sichuan_corpus

IterableDataset({
    features: ['audio', 'label'],
    num_shards: 1
})

In [12]:
shanghai_corpus

IterableDataset({
    features: ['audio', 'gender', 'speaker_id', 'transcription'],
    num_shards: 1
})

In [17]:
next(iter(shanghai_corpus['audio']['path']))

RuntimeError: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6 and 7.
          2. The PyTorch version (2.6.0) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 7: libavutil.so.59: cannot open shared object file: No such file or directory
FFmpeg version 6: libavutil.so.58: cannot open shared object file: No such file or directory
FFmpeg version 5: libavutil.so.57: cannot open shared object file: No such file or directory
FFmpeg version 4: libavutil.so.56: cannot open shared object file: No such file or directory
[end o